## **Informações contidas na base de dados**

**Base 1:**
1. ID do filme
2. título e ano de lançamento

**Base 2:**
1. Cust_Id: ID do customer que fez a avaliação
2. Rating: avaliação (nota)
3. Date: data da avaliação
4. Movie_Id: ID do filme

### Código para tratamento dos bancos dados com Pandas

#### Tratamento do banco de dados da base1 com o nome dos filmes e id de identificação

In [1]:
# Importando o Spark
from pyspark.sql import SparkSession

In [2]:
# Iniciando uma sessao Spark
spark = SparkSession.builder.appName('pyspark_treatment').getOrCreate()

In [3]:
# Importando a biblioteca time para pegar os tempos de execucao
import time

In [4]:
# Dicionario para salvar os tempos de execucao para o dataset movies.csv
execution_time_spk = {}

In [5]:
# Importando a base de dados dos filmes
t1 = time.time()
df_movies = spark.read.csv('movies.csv', sep=';', header=False, inferSchema=True)
tempo_exec = time.time() - t1
execution_time_spk['import_time_csv'] = tempo_exec
print(execution_time_spk)

{'import_time_csv': 3.224440574645996}


In [6]:
# Visualizando a base de dados
df_movies.show()

+---+--------------------+
|_c0|                 _c1|
+---+--------------------+
|  1|(Dinosaur Planet,...|
|  2|(Isle of Man TT 2...|
|  3|   (Character, 1997)|
|  4|(Paula Abdul's Ge...|
|  5|(The Rise and Fal...|
|  6|        (Sick, 1997)|
|  7|       (8 Man, 1992)|
|  8|(What the #$*! Do...|
|  9|(Class of Nuke 'E...|
| 10|     (Fighter, 2001)|
| 11|(Full Frame: Docu...|
| 12|(My Favorite Brun...|
| 13|(Lord of the Ring...|
| 14|(Nature: Antarcti...|
| 15|(Neil Diamond: Gr...|
| 16|   (Screamers, 1996)|
| 17|   (7 Seconds, 2005)|
| 18|(Immortal Beloved...|
| 19|(By Dawn's Early ...|
| 20|(Seeta Aur Geeta,...|
+---+--------------------+
only showing top 20 rows



In [7]:
# Transformando coluna de nomes e anos de lancamento misturados
# Coluna com nome do filme

# Primeiramente precisamos importar o pyspark.sql.functions
from pyspark.sql.functions import *

# Eliminando os parentes dos valores da coluna dos nomes dos filmes e das datas de lancamento
df_movies = df_movies.withColumn("_c1", regexp_replace(df_movies["_c1"], "\(|\)", ""))

# Separando coluna com split pela virgula
df_movies = df_movies.withColumn('splited_column', split(col('_c1'), ','))

# Pegando primeira parte da coluna splitada e criando a coluna com nome do filme
t1 = time.time()
df_movies = df_movies.withColumn('Movie_Name', col('splited_column')[0])
tempo_exec = time.time() - t1
execution_time_spk['new_column_movie'] = tempo_exec
print(execution_time_spk)

{'import_time_csv': 3.224440574645996, 'new_column_movie': 0.007007598876953125}


In [8]:
# Pegando segunda parte da coluna splitada e criando a coluna com ano de lancamento
t1 = time.time()
df_movies = df_movies.withColumn('Release_Year', col('splited_column')[1])
tempo_exec = time.time() - t1
execution_time_spk['new_column_release_year'] = tempo_exec
print(execution_time_spk)

{'import_time_csv': 3.224440574645996, 'new_column_movie': 0.007007598876953125, 'new_column_release_year': 0.007005929946899414}


In [9]:
# Visualizando o dataframe modificado
df_movies.show(5)

+---+--------------------+--------------------+--------------------+------------+
|_c0|                 _c1|      splited_column|          Movie_Name|Release_Year|
+---+--------------------+--------------------+--------------------+------------+
|  1|Dinosaur Planet, ...|[Dinosaur Planet,...|     Dinosaur Planet|        2003|
|  2|Isle of Man TT 20...|[Isle of Man TT 2...|Isle of Man TT 20...|        2004|
|  3|     Character, 1997|  [Character,  1997]|           Character|        1997|
|  4|Paula Abdul's Get...|[Paula Abdul's Ge...|Paula Abdul's Get...|        1994|
|  5|The Rise and Fall...|[The Rise and Fal...|The Rise and Fall...|        2004|
+---+--------------------+--------------------+--------------------+------------+
only showing top 5 rows



In [10]:
# Exclusao da coluna splitada
df_movies = df_movies.drop('splited_column')

In [11]:
# Exclusao da coluna inicial (_c1) com nome e data de lancamento
t1 = time.time()
df_movies = df_movies.drop('_c1')
tempo_exec = time.time() - t1
execution_time_spk['remove_initial_column'] = tempo_exec
print(execution_time_spk)

{'import_time_csv': 3.224440574645996, 'new_column_movie': 0.007007598876953125, 'new_column_release_year': 0.007005929946899414, 'remove_initial_column': 0.004002571105957031}


In [12]:
# Visualização do banco de dados 
df_movies.show(5)

+---+--------------------+------------+
|_c0|          Movie_Name|Release_Year|
+---+--------------------+------------+
|  1|     Dinosaur Planet|        2003|
|  2|Isle of Man TT 20...|        2004|
|  3|           Character|        1997|
|  4|Paula Abdul's Get...|        1994|
|  5|The Rise and Fall...|        2004|
+---+--------------------+------------+
only showing top 5 rows



In [13]:
# Renomeação da coluna de ID do filme com a mesma nomenclatura da base de dados de avaliações.
t1 = time.time()
df_movies = df_movies.withColumnRenamed('_c0', 'Movie_Id')
tempo_exec = time.time() - t1
execution_time_spk['rename_column_movie'] = tempo_exec
print(execution_time_spk)

{'import_time_csv': 3.224440574645996, 'new_column_movie': 0.007007598876953125, 'new_column_release_year': 0.007005929946899414, 'remove_initial_column': 0.004002571105957031, 'rename_column_movie': 0.005005598068237305}


In [14]:
# Informações sobre o dataframe do banco de dados de filmes.
df_movies.printSchema()

root
 |-- Movie_Id: integer (nullable = true)
 |-- Movie_Name: string (nullable = true)
 |-- Release_Year: string (nullable = true)



In [15]:
# Realizando a modificando do tipo de dado da coluna com a data de lançamento.
t1 = time.time()
df_movies = df_movies.withColumn('Release_Year', to_date(df_movies['Release_Year']))
tempo_exec = time.time() - t1
execution_time_spk['change_type_year'] = tempo_exec
print(execution_time_spk)

{'import_time_csv': 3.224440574645996, 'new_column_movie': 0.007007598876953125, 'new_column_release_year': 0.007005929946899414, 'remove_initial_column': 0.004002571105957031, 'rename_column_movie': 0.005005598068237305, 'change_type_year': 0.009007692337036133}


In [16]:
# Informações sobre o dataframe do banco de dados de filmes.
df_movies.printSchema()

root
 |-- Movie_Id: integer (nullable = true)
 |-- Movie_Name: string (nullable = true)
 |-- Release_Year: date (nullable = true)



In [17]:
# Visualização do dataframe com todas as alterações.
df_movies.show(5)

+--------+--------------------+------------+
|Movie_Id|          Movie_Name|Release_Year|
+--------+--------------------+------------+
|       1|     Dinosaur Planet|  2003-01-01|
|       2|Isle of Man TT 20...|  2004-01-01|
|       3|           Character|  1997-01-01|
|       4|Paula Abdul's Get...|  1994-01-01|
|       5|The Rise and Fall...|  2004-01-01|
+--------+--------------------+------------+
only showing top 5 rows



In [18]:
# Verificando se possui dados ausentes.
t1 = time.time()
null_count_col1 = df_movies.filter(df_movies.Movie_Id.isNull()).count()
print(f'Movie_Id\t{null_count_col1}')
null_count_col2 = df_movies.filter(df_movies.Movie_Name.isNull()).count()
print(f'Movie_Name\t{null_count_col2}')
null_count_col3 = df_movies.filter(df_movies.Release_Year.isNull()).count()
print(f'Release_Year\t{null_count_col3}')
tempo_exec = time.time() - t1
execution_time_spk['check_missing_data'] = tempo_exec
print(execution_time_spk)

Movie_Id	0
Movie_Name	0
Release_Year	0
{'import_time_csv': 3.224440574645996, 'new_column_movie': 0.007007598876953125, 'new_column_release_year': 0.007005929946899414, 'remove_initial_column': 0.004002571105957031, 'rename_column_movie': 0.005005598068237305, 'change_type_year': 0.009007692337036133, 'check_missing_data': 0.5204088687896729}


In [19]:
# Verificando se possui linhas duplicadas na base de dados e removendo
t1 = time.time()
df_movies_without_duplicates = df_movies.dropDuplicates()
duplicated_rows = df_movies.count() - df_movies_without_duplicates.count()
tempo_exec = time.time() - t1
execution_time_spk['check_duplicate_lines'] = tempo_exec
print(execution_time_spk)
print(f'Linhas duplicadas: {duplicated_rows}')

{'import_time_csv': 3.224440574645996, 'new_column_movie': 0.007007598876953125, 'new_column_release_year': 0.007005929946899414, 'remove_initial_column': 0.004002571105957031, 'rename_column_movie': 0.005005598068237305, 'change_type_year': 0.009007692337036133, 'check_missing_data': 0.5204088687896729, 'check_duplicate_lines': 0.4919419288635254}
Linhas duplicadas: 0


In [20]:
# Verificando se possui id's duplicados.
t1 = time.time()
df_without_duplitate_id = df_movies.dropDuplicates(subset=['Movie_Id'])
count_df_without_duplitate_id = df_movies.count() - df_without_duplitate_id.count()
tempo_exec = time.time() - t1
execution_time_spk['check_duplicate_lines_ids'] = tempo_exec
print(execution_time_spk)
print(f'IDs duplicados: {count_df_without_duplitate_id}')

{'import_time_csv': 3.224440574645996, 'new_column_movie': 0.007007598876953125, 'new_column_release_year': 0.007005929946899414, 'remove_initial_column': 0.004002571105957031, 'rename_column_movie': 0.005005598068237305, 'change_type_year': 0.009007692337036133, 'check_missing_data': 0.5204088687896729, 'check_duplicate_lines': 0.4919419288635254, 'check_duplicate_lines_ids': 0.30828022956848145}
IDs duplicados: 0


In [21]:
# Verificando se possui filmes com mesmo nome e mesmo ano de lançamento.
t1 = time.time()
df_group = df_movies.groupBy(df_movies.Movie_Name, df_movies.Release_Year).agg(count('*').alias('Count'))
df_duplicates = df_group.filter(df_group['Count'] > 1)
tempo_exec = time.time() - t1
execution_time_spk['check_duplicate_lines_movie_year'] = tempo_exec
print(execution_time_spk)
df_duplicates.show(5)

{'import_time_csv': 3.224440574645996, 'new_column_movie': 0.007007598876953125, 'new_column_release_year': 0.007005929946899414, 'remove_initial_column': 0.004002571105957031, 'rename_column_movie': 0.005005598068237305, 'change_type_year': 0.009007692337036133, 'check_missing_data': 0.5204088687896729, 'check_duplicate_lines': 0.4919419288635254, 'check_duplicate_lines_ids': 0.30828022956848145, 'check_duplicate_lines_movie_year': 0.018016338348388672}
+----------+------------+-----+
|Movie_Name|Release_Year|Count|
+----------+------------+-----+
| Dr. Quinn|  1993-01-01|    2|
+----------+------------+-----+



In [22]:
# Obtendo tabela com os registros do filme repetido
t1 = time.time()
df_duplicates_final = df_duplicates.join(df_movies, \
    on=((df_duplicates.Movie_Name == df_movies.Movie_Name) & \
        (df_duplicates.Release_Year == df_movies.Release_Year)))
tempo_exec = time.time() - t1
execution_time_spk['duplicate_line_dr_quinn'] = tempo_exec
print(execution_time_spk)
df_duplicates_final.show()

{'import_time_csv': 3.224440574645996, 'new_column_movie': 0.007007598876953125, 'new_column_release_year': 0.007005929946899414, 'remove_initial_column': 0.004002571105957031, 'rename_column_movie': 0.005005598068237305, 'change_type_year': 0.009007692337036133, 'check_missing_data': 0.5204088687896729, 'check_duplicate_lines': 0.4919419288635254, 'check_duplicate_lines_ids': 0.30828022956848145, 'check_duplicate_lines_movie_year': 0.018016338348388672, 'duplicate_line_dr_quinn': 0.02902698516845703}
+----------+------------+-----+--------+----------+------------+
|Movie_Name|Release_Year|Count|Movie_Id|Movie_Name|Release_Year|
+----------+------------+-----+--------+----------+------------+
| Dr. Quinn|  1993-01-01|    2|    4005| Dr. Quinn|  1993-01-01|
| Dr. Quinn|  1993-01-01|    2|     350| Dr. Quinn|  1993-01-01|
+----------+------------+-----+--------+----------+------------+



Observação: Apesar de existir dois registros com o mesmo nome (Dr. Quinn) e mesmo ano de lançamento (1993), realizando uma busca na internet, foi verificado que no ano de 1993 foram lançados duas temporadas de Dr. Quinn. Dessa forma, os registros estão se referindo a temporadas diferentes, não sendo considerado um registro duplicado.

#### Tratamento do banco de dados da base2 com as avaliações dos usuários

In [23]:
# Criando dicionario para salvar tempos de execucao
execution_time2_spk = {}

In [24]:
# Importando a segunda base de dados
t1 = time.time()
df_customers_rating = spark.read.csv('customers_rating.csv', sep=';', header=True, inferSchema=True)
tempo_exec = time.time() - t1
execution_time2_spk['import_time_csv'] = tempo_exec
print(execution_time2_spk)

{'import_time_csv': 4.452035188674927}


In [25]:
# Visualizacao do dataframe
df_customers_rating.show(5)

+-------+------+-------------------+--------+
|Cust_Id|Rating|               Date|Movie_Id|
+-------+------+-------------------+--------+
|1488844|   3.0|2005-09-06 00:00:00|       1|
| 822109|   5.0|2005-05-13 00:00:00|       1|
| 885013|   4.0|2005-10-19 00:00:00|       1|
|  30878|   4.0|2005-12-26 00:00:00|       1|
| 823519|   3.0|2004-05-03 00:00:00|       1|
+-------+------+-------------------+--------+
only showing top 5 rows



In [26]:
# Verificando se possui dados ausentes.
t1 = time.time()
null_count_col1 = df_customers_rating.filter(df_customers_rating.Cust_Id.isNull()).count()
print(f'Cust_Id\t{null_count_col1}')
null_count_col2 = df_customers_rating.filter(df_customers_rating.Rating.isNull()).count()
print(f'Rating\t{null_count_col2}')
null_count_col3 = df_customers_rating.filter(df_customers_rating.Date.isNull()).count()
print(f'Date\t{null_count_col3}')
null_count_col4 = df_customers_rating.filter(df_customers_rating.Movie_Id.isNull()).count()
print(f'Movie_Id\t{null_count_col4}')
tempo_exec = time.time() - t1
execution_time2_spk['check_missing_data'] = tempo_exec
print(execution_time2_spk)

Cust_Id	0
Rating	0
Date	0
Movie_Id	0
{'import_time_csv': 4.452035188674927, 'check_missing_data': 7.960012674331665}


In [27]:
# Informações sobre o dataframe. 
df_customers_rating.printSchema()

root
 |-- Cust_Id: integer (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Movie_Id: integer (nullable = true)



In [28]:
# Realizando a converteção da coluna de Date para datetime. Importante trabalhar com essa coluna no formato de data.
t1 = time.time()
df_customers_rating = df_customers_rating.withColumn('Date', to_date(df_customers_rating['Date']))
tempo_exec = time.time() - t1
execution_time2_spk['change_type_date'] = tempo_exec
print(execution_time2_spk)

{'import_time_csv': 4.452035188674927, 'check_missing_data': 7.960012674331665, 'change_type_date': 0.005509138107299805}


In [29]:
# Visualizacao do dataframe
df_customers_rating.show(5)

+-------+------+----------+--------+
|Cust_Id|Rating|      Date|Movie_Id|
+-------+------+----------+--------+
|1488844|   3.0|2005-09-06|       1|
| 822109|   5.0|2005-05-13|       1|
| 885013|   4.0|2005-10-19|       1|
|  30878|   4.0|2005-12-26|       1|
| 823519|   3.0|2004-05-03|       1|
+-------+------+----------+--------+
only showing top 5 rows



In [30]:
# Confirmando a mudança da coluna Date para date
df_customers_rating.printSchema()

root
 |-- Cust_Id: integer (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Movie_Id: integer (nullable = true)



In [31]:
# Verificação se possui avaliações duplicadas de clientes, 
# considerando mesmo id do cliente, id do filme e avaliação
t1 = time.time()
df2_group = df_customers_rating.groupBy(df_customers_rating.Cust_Id, \
    df_customers_rating.Movie_Id, df_customers_rating.Rating).agg(count('*').alias('Count'))
df2_duplicates = df2_group.filter(df2_group['Count'] > 1)
tempo_exec = time.time() - t1
execution_time2_spk['check_duplicate_lines_movie_year'] = tempo_exec
print(execution_time2_spk)
df2_duplicates.show(5)

{'import_time_csv': 3.224440574645996, 'new_column_movie': 0.007007598876953125, 'new_column_release_year': 0.007005929946899414, 'remove_initial_column': 0.004002571105957031, 'rename_column_movie': 0.005005598068237305, 'change_type_year': 0.009007692337036133, 'check_missing_data': 0.5204088687896729, 'check_duplicate_lines': 0.4919419288635254, 'check_duplicate_lines_ids': 0.30828022956848145, 'check_duplicate_lines_movie_year': 0.010524749755859375, 'duplicate_line_dr_quinn': 0.02902698516845703}
+-------+--------+------+-----+
|Cust_Id|Movie_Id|Rating|Count|
+-------+--------+------+-----+
+-------+--------+------+-----+



### **Criando um novo dataframe com join do df_movies e df_customers_rating**

In [32]:
df_customers_rating.printSchema()
df_movies.printSchema()

root
 |-- Cust_Id: integer (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Movie_Id: integer (nullable = true)

root
 |-- Movie_Id: integer (nullable = true)
 |-- Movie_Name: string (nullable = true)
 |-- Release_Year: date (nullable = true)



In [33]:
# Como a análise será realizada apenas com os filmes que possuem avaliações de clientes, 
# será realizado um join considerando a interseção entre as tabelas.
t1 = time.time()
df_join_movies_rating = df_movies.join(df_customers_rating, \
    on=(df_movies.Movie_Id == df_customers_rating.Movie_Id), \
        how='inner').drop('Movie_Id')
tempo_exec = time.time() - t1
execution_time2_spk['merge_tables'] = tempo_exec
print(execution_time2_spk)

{'import_time_csv': 4.452035188674927, 'check_missing_data': 7.960012674331665, 'change_type_date': 0.005509138107299805, 'merge_tables': 0.006005287170410156}


In [34]:
# Visualizacao do dataframe
df_join_movies_rating.show(5)

+---------------+------------+-------+------+----------+
|     Movie_Name|Release_Year|Cust_Id|Rating|      Date|
+---------------+------------+-------+------+----------+
|Dinosaur Planet|  2003-01-01|1488844|   3.0|2005-09-06|
|Dinosaur Planet|  2003-01-01| 822109|   5.0|2005-05-13|
|Dinosaur Planet|  2003-01-01| 885013|   4.0|2005-10-19|
|Dinosaur Planet|  2003-01-01|  30878|   4.0|2005-12-26|
|Dinosaur Planet|  2003-01-01| 823519|   3.0|2004-05-03|
+---------------+------------+-------+------+----------+
only showing top 5 rows



In [35]:
# Quantidade de registros no dataframe
df_join_movies_rating.count()

24053764

In [36]:
"""
# Load dos dados (salvamento)
# Delta
t1 = time.time()
df_join_movies_rating.write.format("delta").mode("overwrite").save('./final_data.delta')
tempo_exec = time.time() - t1
execution_time2_spk['final_load_delta'] = tempo_exec
print(execution_time2_spk)
"""

'\n# Load dos dados (salvamento)\n# Delta\nt1 = time.time()\ndf_join_movies_rating.write.format("delta").mode("overwrite").save(\'./final_data.delta\')\ntempo_exec = time.time() - t1\nexecution_time2_spk[\'final_load_delta\'] = tempo_exec\nprint(execution_time2_spk)\n'

In [37]:
"""
# Load dos dados (salvamento)
# parquet
t1 = time.time()
df_join_movies_rating.write.format("parquet").mode("overwrite").save('./final_data.parquet')
tempo_exec = time.time() - t1
execution_time2_spk['final_load_parquet'] = tempo_exec
print(execution_time2_spk)
"""

'\n# Load dos dados (salvamento)\n# parquet\nt1 = time.time()\ndf_join_movies_rating.write.format("parquet").mode("overwrite").save(\'./final_data.parquet\')\ntempo_exec = time.time() - t1\nexecution_time2_spk[\'final_load_parquet\'] = tempo_exec\nprint(execution_time2_spk)\n'

In [38]:
"""
# Load dos dados (salvamento)
# csv
t1 = time.time()
df_join_movies_rating.write.format("csv").mode("overwrite").save('./final_data.csv')
tempo_exec = time.time() - t1
execution_time2_spk['final_load_csv'] = tempo_exec
print(execution_time2_spk)
"""

'\n# Load dos dados (salvamento)\n# csv\nt1 = time.time()\ndf_join_movies_rating.write.format("csv").mode("overwrite").save(\'./final_data.csv\')\ntempo_exec = time.time() - t1\nexecution_time2_spk[\'final_load_csv\'] = tempo_exec\nprint(execution_time2_spk)\n'

In [39]:
# Salvar os dicionarios com os tempos de execucao para criar o grafico (formato json)
import json

with open('execution_time_spk.json', 'w') as f:
    json.dump(execution_time_spk, f)

with open('execution_time2_spk.json', 'w') as f:
    json.dump(execution_time2_spk, f)